In [3]:
pip install langgraph


   ------------- -------------------------- 2/6 [langgraph-sdk]
   -------------------- ------------------- 3/6 [langgraph-checkpoint]
   --------------------------------- ------ 5/6 [langgraph]
   --------------------------------- ------ 5/6 [langgraph]
   --------------------------------- ------ 5/6 [langgraph]
   --------------------------------- ------ 5/6 [langgraph]
   --------------------------------- ------ 5/6 [langgraph]
   --------------------------------- ------ 5/6 [langgraph]
   --------------------------------- ------ 5/6 [langgraph]
   ---------------------------------------- 6/6 [langgraph]

Note: you may need to restart the kernel to use updated packages.


In [16]:
from langchain_community.utilities import SQLDatabase

from langchain_ollama import OllamaLLM
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")

# -----------------------------
# 1. Build connection string
# -----------------------------
# Replace YOUR_SERVER with your SQL Server Express instance name
# Replace HRDatabase with your database name
# Use trusted connection (Windows Auth) or add UID/PWD for SQL auth
connection_string = (
    "mssql+pyodbc:///?odbc_connect="
    "Driver={ODBC Driver 17 for SQL Server};"
    "Server=LAPTOP-E5BJ5PM9;"
    "Database=HRDatabase;"
    "Trusted_Connection=yes;"
)

# -----------------------------
# 2. Load SQLDatabase
# -----------------------------
db = SQLDatabase.from_uri(connection_string)

In [17]:
print(db.get_usable_table_names())

['Employees']


In [18]:
db.run("SELECT * FROM Employees;")

"[(1, 'Alice Johnson', 'Python, React, AWS', 5, 'E-commerce Platform, Healthcare Dashboard', 'available'), (3, 'Alice Johnson', 'Python, React, AWS', 5, 'E-commerce Platform, Healthcare Dashboard', 'available'), (4, 'Bob Smith', 'Java, Spring Boot, MySQL', 7, 'Banking System, Loan Processing App', 'unavailable'), (5, 'Catherine Lee', 'C#, .NET, Azure', 4, 'Hospital Management, CRM Tool', 'available'), (6, 'David Kumar', 'Python, Django, PostgreSQL', 3, 'Inventory System, Chatbot', 'available'), (7, 'Emma Williams', 'JavaScript, Node.js, MongoDB', 6, 'Real-time Chat App, Food Delivery Platform', 'unavailable'), (8, 'Frank Miller', 'React Native, Firebase, Kotlin', 2, 'Mobile Wallet, Fitness Tracker', 'available'), (9, 'Grace Chen', 'Python, TensorFlow, SQL', 5, 'Healthcare AI Model, Fraud Detection', 'available'), (10, 'Henry Adams', 'PHP, Laravel, MySQL', 8, 'E-learning Platform, Blogging CMS', 'unavailable'), (11, 'Isabella Rossi', 'Ruby, Rails, PostgreSQL', 4, 'Event Booking System, 

In [19]:
from typing_extensions import TypedDict


class State(TypedDict):
    question: str
    query: str
    result: str
    answer: str

In [20]:
import getpass
import os

if not os.environ.get("GROQ_API_KEY"):
  os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

from langchain.chat_models import init_chat_model

llm = init_chat_model("llama-3.3-70b-versatile", model_provider="groq")

In [31]:
from langchain_core.prompts import ChatPromptTemplate

system_message = """
Given an input question, create a syntactically correct {dialect} query to
run to help find the answer. Unless the user specifies in his question a
specific number of examples they wish to obtain. You can order the results by a relevant column to
return the most interesting examples in the database.

Never query for all the columns from a specific table, only ask for a the
few relevant columns given the question.

Pay attention to use only the column names that you can see in the schema
description. Be careful to not query for columns that do not exist. Also,
pay attention to which column is in which table.

Only use the following tables:
{table_info}
"""

user_prompt = "Question: {input}"

query_prompt_template = ChatPromptTemplate(
    [("system", system_message), ("user", user_prompt)]
)

for message in query_prompt_template.messages:
    message.pretty_print()

================================ System Message ================================


Given an input question, create a syntactically correct {dialect} query to
run to help find the answer. Unless the user specifies in his question a
specific number of examples they wish to obtain. You can order the results by a relevant column to
return the most interesting examples in the database.

Never query for all the columns from a specific table, only ask for a the
few relevant columns given the question.

Pay attention to use only the column names that you can see in the schema
description. Be careful to not query for columns that do not exist. Also,
pay attention to which column is in which table.

Only use the following tables:
{table_info}

================================ Human Message =================================

Question: {input}


In [22]:
from typing_extensions import Annotated
from typing import TypedDict

class QueryOutput(TypedDict):
    """Generated SQL query."""

    query: Annotated[str, ..., "Syntactically valid SQL query."]


def write_query(state: State):
    """Generate SQL query to fetch information."""
    prompt = query_prompt_template.invoke(
        {
            "dialect": db.dialect,
            "top_k": 10,
            "table_info": db.get_table_info(),
            "input": state["question"],
        }
    )
    structured_llm = llm.with_structured_output(QueryOutput)
    result = structured_llm.invoke(prompt)
    return {"query": result["query"]}

In [23]:
write_query({"question": "How many Employees are there?"})

{'query': 'SELECT COUNT(EmployeeID) FROM Employees'}

In [24]:
from langchain_community.tools.sql_database.tool import QuerySQLDatabaseTool


def execute_query(state: State):
    """Execute SQL query."""
    execute_query_tool = QuerySQLDatabaseTool(db=db)
    return {"result": execute_query_tool.invoke(state["query"])}

In [25]:
execute_query({"query": "SELECT COUNT(EmployeeID) FROM Employees;"})

{'result': '[(21,)]'}

In [26]:
def generate_answer(state: State):
    """Answer question using retrieved information as context."""
    prompt = (
        "Given the following user question, corresponding SQL query, "
        "and SQL result, answer the user question.\n\n"
        f"question: {state['question']}\n"
        f"SQL Query: {state['query']}\n"
        f"SQL Result: {state['result']}"
    )
    response = llm.invoke(prompt)
    return {"answer": response.content}

In [27]:
from langgraph.graph import START, StateGraph

graph_builder = StateGraph(State).add_sequence(
    [write_query, execute_query, generate_answer]
)
graph_builder.add_edge(START, "write_query")
graph = graph_builder.compile()

In [28]:
# from IPython.display import Image, display

# display(Image(graph.get_graph().draw_mermaid_png()))

In [29]:
for step in graph.stream(
    {"question": "How many employees are unavailable?"}, stream_mode="updates"
):
    print(step)

{'write_query': {'query': "SELECT COUNT(EmployeeID) FROM Employees WHERE Availability = 'unavailable'"}}
{'execute_query': {'result': '[(7,)]'}}
{'generate_answer': {'answer': 'There are 7 employees who are unavailable.'}}


In [27]:
for step in graph.stream(
    {"question": "How many employees have skill in Python?"}, stream_mode="updates"
):
    print(step)

{'write_query': {'query': "SELECT COUNT(EmployeeID) FROM Employees WHERE Skills LIKE '%Python%'"}}
{'execute_query': {'result': '[(6,)]'}}
{'generate_answer': {'answer': 'There are 6 employees who have a skill in Python.'}}


In [34]:
for step in graph.stream(
    {"question": "I need someone experienced in healthcare project?"}, stream_mode="updates"
):
    print(step)

{'write_query': {'query': "SELECT Name, ExperienceYears, Skills, Availability FROM Employees WHERE Projects LIKE '%Healthcare%' ORDER BY ExperienceYears DESC"}}
{'execute_query': {'result': "[('Alice Johnson', 5, 'Python, React, AWS', 'available'), ('Alice Johnson', 5, 'Python, React, AWS', 'available'), ('Grace Chen', 5, 'Python, TensorFlow, SQL', 'available')]"}}
{'generate_answer': {'answer': 'Based on the SQL result, there are two employees with experience in healthcare projects: \n\n1. Alice Johnson - with 5 years of experience and skills in Python, React, and AWS. She is available.\n2. Grace Chen - with 5 years of experience and skills in Python, TensorFlow, and SQL. She is available.\n\nNote that Alice Johnson appears twice in the result, possibly due to duplicate entries in the database. However, for the purpose of answering the user question, we can consider her as one candidate. \n\nBoth Alice Johnson and Grace Chen have the same amount of experience, so it ultimately depends